# TSMixer

In [ ]:
!pip install pytorch-tsmixer

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtsmixer import TSMixer
import pandas as pd 
import sys
from torch.utils.data import DataLoader, TensorDataset


ts = pd.read_csv('covidDataWithSentiment.csv')

# Drop cumulative datas?
cumulative_clmsn = []
for clmn_name in ts:
    if "Cumulative" in clmn_name:
        cumulative_clmsn.append(clmn_name)
for entry in cumulative_clmsn:
    ts.drop(columns=entry, inplace=True)

# Drop date
data = ts.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8', 'uint8'])
data = data.fillna(0)

# Model parameters
sequence_length = 14
prediction_length = 5
input_channels = len(data.columns) - 1
output_channels = 1


X_train = torch.empty(0, sequence_length, input_channels)
y_train = torch.empty(0, prediction_length, output_channels)

for start in range(1, len(data) - sequence_length - prediction_length + 1):
    train_inst = torch.tensor(data.iloc[start:start + sequence_length, :-1].values, dtype=torch.float32)
    label_inst = torch.tensor(data.iloc[start + sequence_length : start + sequence_length + prediction_length, -1:].values, dtype=torch.float32)    

    X_train = torch.cat((X_train, train_inst.unsqueeze(0)), dim = 0)
    y_train = torch.cat((y_train, label_inst.unsqueeze(0)), dim = 0)

print(X_train.shape)
print(y_train.shape)


dataset = TensorDataset(X_train, y_train)
# Create a DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)



# Create the TSMixer model
model = TSMixer(sequence_length, prediction_length, input_channels, output_channels)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Training loop
num_epochs = 100000
for epoch in range(num_epochs):
    model.train()
    for X, y in dataloader:
        # Extract a single batch from the dataset   
                
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(X) 
        loss = criterion(outputs, y)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training complete")

torch.Size([401, 14, 12])
torch.Size([401, 5, 1])
Epoch [1/100000], Loss: 4168795.5000
Epoch [2/100000], Loss: 4168856.0000
Epoch [3/100000], Loss: 4168802.2500
Epoch [4/100000], Loss: 4168820.7500
Epoch [5/100000], Loss: 4168730.7500
Epoch [6/100000], Loss: 4168756.2500
Epoch [7/100000], Loss: 4168692.7500
Epoch [8/100000], Loss: 4168633.2500
Epoch [9/100000], Loss: 4168600.7500
Epoch [10/100000], Loss: 4168500.2500
Epoch [11/100000], Loss: 4168541.7500
Epoch [12/100000], Loss: 4168418.2500
Epoch [13/100000], Loss: 4168280.7500
Epoch [14/100000], Loss: 4168336.2500
Epoch [15/100000], Loss: 4168222.7500
Epoch [16/100000], Loss: 4168098.7500
Epoch [17/100000], Loss: 4167901.2500
Epoch [18/100000], Loss: 4167960.7500
Epoch [19/100000], Loss: 4167585.2500
Epoch [20/100000], Loss: 4167586.0000
Epoch [21/100000], Loss: 4167400.0000
Epoch [22/100000], Loss: 4167344.5000
Epoch [23/100000], Loss: 4167079.2500
Epoch [24/100000], Loss: 4166797.2500
Epoch [25/100000], Loss: 4166606.0000
Epoch [26

KeyboardInterrupt: 